In [303]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
pd.set_option('display.max_columns', None)
%matplotlib inline

In [304]:
attack_file = pd.read_csv('attacks.csv', encoding= "ISO-8859-1")
f = attack_file.copy()
orig_shape = attack_file.shape

In [305]:
f.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-18,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-18,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-18,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-18,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [306]:
#f[f['Unnamed: 22'].notna() == True]
#f[f['Unnamed: 23'].notna() == True]
# Going to drop these columns since they are practically empty and have no real meaning
#f.drop(columns=['Unnamed: 22','Unnamed: 23'], axis=1,inplace=True)

In [307]:
f.shape

(25723, 24)

In [308]:
# Clean up column names

strip_dict = {}
for i,j in enumerate(f.columns):
    strip_dict[j] = j.strip()

f.rename(columns= strip_dict,inplace = True)
f.rename(columns={'Fatal (Y/N)':'Fatal','Investigator or Source':'Source'},inplace=True)

In [309]:
# identify high NaN lines and replace useless column
nun_nan = []

for fila in f.itertuples():
    
    check=[]
    
    for e in fila:
        
        check.append(pd.isna(e))
        
    nun_nan.append(sum(check))
f['Unnamed: 22'] = nun_nan
f.rename(columns={'Unnamed: 22':'NaN Columns'},inplace=True)


In [310]:
#identify nulls
nullity = pd.DataFrame([f.notna().sum(), f.isnull().sum()], ['Not Nulls','Nulls']).T
nullity.sort_values(by='Nulls',inplace=True)
nullity

,Not Nulls,Nulls
NaN Columns,25723,0
Case Number,8702,17021
original order,6309,19414
Date,6302,19421
Case Number.2,6302,19421
Case Number.1,6302,19421
href,6302,19421
pdf,6302,19421
href formula,6301,19422
Year,6300,19423


In [311]:
# eliminate NaN >10 rows
f.drop(index = f[f['NaN Columns']>10].index, inplace=True)
f.shape

(6302, 24)

In [312]:
# Identify strange years
f[['Year','Date','Time','Case Number']][f['Year'].isna()]
f[['Year','Date','Time','Case Number']][f['Year']<1500]

,Year,Date,Time,Case Number
6174,500.0,Circa 500 A.D.,NaN,0500.00.00
6175,77.0,77 A.D.,NaN,0077.00.00
6176,5.0,Ca. 5 A.D.,NaN,0005.00.00
6177,0.0,Ca. 214 B.C.,NaN,0.0214
6178,0.0,Ca. 336.B.C..,NaN,0.0336
...,...,...,...,...
6297,0.0,Before 1903,NaN,ND.0005
6298,0.0,Before 1903,NaN,ND.0004
6299,0.0,1900-1905,NaN,ND.0003
6300,0.0,1883-1889,NaN,ND.0002


In [313]:
#replace NaNs with correct digit
f.at[187,'Year'] = 2017
f.at[6079,'Year'] = 1836
print(f.iloc[187]['Year'],f.iloc[6079]['Year'])

#remove Non-well-defined years and unreliable
f.drop(index = f[f['Year']<1500].index,inplace = True)
print(f[['Year','Date','Time','Case Number']][(f['Year']<1500) | f['Year'].isna()])

2017.0 1836.0
Empty DataFrame
Columns: [Year, Date, Time, Case Number]
Index: []


In [314]:
#Explore Case Number
f[f['Case Number'].isna()] # 1 Nan that can be fixed
bad_index = f.index[f['Case Number'].isna()][0]
f.at[bad_index,'Case Number'] = '1905.09.05.R'

# find bad values
non_standard = []

for fila in f['Case Number']:
    try:
        nothing = re.findall('\d{4}.\d{2}.\d{2}',fila)[0]
    except:
        non_standard.append(fila,)

len(non_standard) # only 1 value

# correct bad case number index
bad_index = f['Case Number'].index[f['Case Number'] == non_standard[0]][0]
f.at[ bad_index , 'Case Number' ] = '1864.05.00'


In [315]:
# New date columns based on Case number
dates = []
for fila in f['Case Number']:
    dates.append(re.findall('(\d{4}).(\d{2}).(\d{2})?(.*)',fila))
new_years = []
new_months = []
new_days = []
extra = []

for date in dates:
    new_years.append(date[0][0])
    new_months.append(date[0][1])
    new_days.append(date[0][2])
    extra.append(date[0][3])


f['New Year'] = new_years
f['New Month'] = new_months
f['New Day'] = new_days
f['Notes'] = extra
f['DateConc'] = f['New Year'] + '-' + f['New Month'] + '-'+ f['New Day']



In [316]:
# Notes - Reporteds
f['Reported'] = np.where(f['Notes'].str.contains('R'),'Y','N')
f.shape

(6174, 30)

In [317]:
# Check for duplicates
f.duplicated().sum() == 0 # True, so now we can duplicate rows with several cases
f['Many'] = 0

g = pd.concat([f,f[f['Notes'] =='.a & b'],f[f['Notes'] =='.R.a & b']]) # Concatenate those which are duped within the row
g.reset_index(inplace=True,drop=True) #reset the index

# Count duplicates sequentially to identify separate cases
for i in range(len(g)):
    g.at[ i , 'Many' ] = g['DateConc'][(g['DateConc'].duplicated() == True) & (g.index <= i) & (g['DateConc'] == g['DateConc'][i])].count()+1



In [318]:
# find anomalies
g['New Month'].unique() # month = 94?
g['New Month'] = g['New Month'].replace('94','04')
g['New Day'].unique() # day = 38?
g['New Day'] = g['New Day'].replace('38','28')

In [319]:
# New case number is cleaner and identifies cases in same day
g['Case Number'] = g['New Year'] + '.' + g['New Month'] + '.' + g['New Day'] + ', ' + g['Many'].astype(str)


In [320]:

g.loc[(g['New Day'] != '00'),'Approx Day'] = g['New Day']
g.loc[(g['New Day'] == '00'),'Approx Day'] = '01'
g.loc[(g['New Month'] != '00'),'Approx Month'] = g['New Month']
g.loc[(g['New Month'] == '00'),'Approx Month'] = '01'
# Create new date Column
g['Date Approx'] = pd.to_datetime(g['Approx Day'] +'-'+g['Approx Month']+'-'+g['New Year'], infer_datetime_format=True,errors='coerce')
g.drop(g[g['Date Approx'].isna()].index,inplace=True) # Remove really low dates that pandas doesnt accept
g.shape


(6166, 34)

In [321]:
g[g['New Month'] == '00'].count() # 391 entries don't have a definite month and values range wildly, so will delete
g.drop(g[g['New Month'] == '00'].index,inplace=True)
g.shape
g[g['New Day'] == '00'].count() #346 entries but could still be useful

Case Number       346
Date              346
Year              346
Type              346
Country           339
Area              294
Location          280
Activity          297
Name              320
Sex               278
Age                88
Injury            343
Fatal             313
Time               48
Species           168
Source            345
pdf               346
href formula      346
href              346
Case Number.1     346
Case Number.2     346
original order    346
NaN Columns       346
Unnamed: 23         0
New Year          346
New Month         346
New Day           346
Notes             346
DateConc          346
Reported          346
Many              346
Approx Day        346
Approx Month      346
Date Approx       346
dtype: int64

In [322]:
# Year column has no bad values
for i in g.Year:
    try:
        i = int(i)
    except:
        print(i)


In [323]:
# small test to check if years actually coincide

g['test column'] = 0
for i in range(len(g)):
    g['test column'] = int(g['New Year'].iloc[i]) - int(g['Year'].iloc[i])
g['test column'].unique() # only 0, so works, can now remove test column

# Change Date column to approximate Date column which contains the proper values with "reported" removed and added 1 to every unknown day
# Remove excess year/month/day columns



g.drop(columns=['test column','New Year','Date','Unnamed: 23', 'Approx Day','Approx Month','DateConc','Notes','New Day','New Month'], axis=1,inplace=True) 
g.rename(columns={'Date Approx':'Date'},inplace=True)

In [324]:
# Clean up the rest of the columns
for i in g.columns:
    try:
        g[i] = g[i].str.strip()
    except:
        continue


In [325]:
# Clean up Age column to get rid of uncertain ages

for i in range(len(g)):
    try:
        x = int(g['Age'].iloc[i])
        if x == 0:
            g.at[i, 'Age'] = np.nan
        else:
            g.at[i, 'Age'] = x
        
    except:
        g.at[i, 'Age'] = np.nan
g.drop(index = g[g['Age'] == '2½'].index,inplace = True)

g['Age'].isna().sum()

2740

In [326]:
# Clean up Sex column to get rid of uncertain sexes
g['Sex'].unique()
for i in g[(g['Sex'] != 'F') & (g['Sex'] != 'M')].index:

    g.at[i, 'Sex'] = np.nan


In [327]:
# clean up NaNs, empty rows, and number types

g.drop(g[g['Case Number'].isna()].index,inplace=True) 


g['Age'].fillna(0,inplace=True)
g['Age']=g['Age'].apply(pd.to_numeric, errors='coerce')
g['Year']=g['Year'].apply(pd.to_numeric, errors='coerce')
g.astype({'Age':'int','Year':'int'})

g.fillna('Unknown',inplace=True)

In [328]:
g['Country'] = g['Country'].str.upper() # Standardize

g.drop(g[g['Country'].str.contains('/')].index,inplace=True) # Clean uncertain countries


In [329]:
g['Type'].unique()
for i in g[(g['Type'] == 'Boat') | (g['Type'] == 'Boatomg')].index: # Standardize errors

    g.at[i, 'Type'] = 'Boating'

g.drop(g[g['Type'] == 'Questionable'].index,inplace=True) # Get rid of questionable data
g[g['Type'] == 'Invalid'].head() # 508 entries of questionable shark involvement, will keep for now to compare against sex data



,Case Number,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,NaN Columns,Reported,Many,Date
2,"2018.06.09, 1",2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,Unknown,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,3.0,N,1.0,2018-06-09
13,"2018.05.13, 2",2018.0,Invalid,ENGLAND,Cornwall,Off Land's End,Fishing,Max Berryman,M,21,Injured by teeth of a dead porbeagle shark he ...,N,08h15,Invalid incident,"K. McMurray, TrackingSharks.com",2018.05.13.a-Berryman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.13.a,2018.05.13.a,6290.0,2.0,N,2.0,2018-05-13
31,"2018.04.10, 1",2018.0,Invalid,BRAZIL,Alagoas,"Praia de Sauaçuhy, Maceió",Fishing,Josias Paz,M,56,Injury to ankle from marine animal trapped in ...,N,Unknown,Shark involvement not confirmed,"K. McMurray, TrackingSharks.com",2018.04.10.R-Paz.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.04.10.R,2018.04.10.R,6272.0,3.0,Y,1.0,2018-04-10
43,"2018.02.14, 1",2018.0,Invalid,AUSTRALIA,Queensland,Mooloolaba Beach,Swimming,Sharna Babd,F,0,"Collision / No injury, no attack",N,18h00,Possibly a wobbegong,"Sunshine Coast Daily, 2/15/2018",2018.02.14-Babd.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.02.14,2018.02.14,6260.0,3.0,N,1.0,2018-02-14
44,"2018.02.11, 1",2018.0,Invalid,BRAZIL,"Boi Island, Victoria",Espirito Santo,Cleaning fish,Rosalida Souza,F,46,Lacerations to 4 toes of right foot,N,Unknown,"Injury believed caused by an eel, not a shark",TrackingSharks.com,2018.02.11-Rosilda.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.02.11,2018.02.11,6259.0,3.0,N,1.0,2018-02-11


In [330]:
g.to_csv('cleaned.csv',index=False)

### Analysis

In [364]:
h = g.copy()
h.head()
h[h['Type'] == 'Unknown']

,Case Number,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,NaN Columns,Reported,Many,Date
85,"2017.09.15, 2",2017.0,Unknown,SAMOA,Upolu Island,Nofoalii,Fishing,male,M,0,Injuries to hands and legs,N,Night,Unknown,"Samoa Observer, 9/16/2017",2017.09.15.a-Samoa.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.09.15.a,2017.09.15.a,6218.0,5.0,N,2.0,2017-09-15
382,"2015.07.27, 1",2015.0,Unknown,AUSTRALIA,Victoria,Tyrendarra Beach,Surfing,male,M,40,Injury to hand,Unknown,Unknown,Unknown,Unknown,2015.07.27-Victoria.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.07.27,2015.07.27,5921.0,7.0,N,1.0,2015-07-27
4867,"1936.09.11, 1",1936.0,Unknown,VIETNAM,Unknown,Saigon,Wreck of a sampam,8 crew,M,0,FATAL,Y,Unknown,Unknown,"Lansing State Journal, 9/11/1936",1936.09.11-Saigon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1936.09.11.R,1936.09.11.R,1436.0,7.0,Y,1.0,1936-09-11
5705,"1890.03.03, 1",1890.0,Unknown,CEYLON,Unknown,Unknown,Diving,a pearl diver,M,0,FATAL,Y,Unknown,Unknown,"The Guardian, 3/3/1890",1890.03.03.R-Ceylon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1890.03.03.R,1890.03.03.R,598.0,8.0,Y,1.0,1890-03-03


In [409]:
#How does type of attack correlate with Sex

k = h.pivot_table(index=['Sex'], columns={'Type'},aggfunc={'Type':'count'})['Type']
k.drop(columns = 'Unknown',inplace= True)
k['Total'] = 0
for i in k.columns[k.columns != 'Total']:
    k['Total'] += k[i]


for i in k.columns[k.columns != 'Total']:
    k['%' + i] = 100*k[i]/k['Total']
k[k.index != 'Unknown']

Type,Boating,Invalid,Provoked,Sea Disaster,Unprovoked,Total,%Boating,%Invalid,%Provoked,%Sea Disaster,%Unprovoked
Sex,,,,,,,,,,,
F,9.0,58.0,22.0,8.0,497.0,594.0,1.515152,9.764310,3.703704,1.346801,83.670034
M,110.0,382.0,465.0,147.0,3554.0,4658.0,2.361529,8.200945,9.982825,3.155861,76.298841


In [410]:
# Women are less likely to provoke a shark, and have more unprovoked accidents
# Men are more likely to have been in sea disasters and boating accidents
# Both men and women have a similar proportion of invalid attacks, meaning they exaggerate at a similar rate, with women slightly more

In [426]:
# Let's look at type of attack  through the years

#bin function to control our output
def bins(size,minyear,maxyear):
    bin =[minyear]
    while minyear<maxyear:
        minyear += size
        bin.append(minyear)
    return bin

# Output table depending on what values we wish to see
def type_by_year(size=50,minyear=1700,maxyear=2019):
    

    groups = h.groupby([pd.cut(h['Year'], bins(size,minyear,maxyear)), 'Type'])
    l = groups.size().unstack(fill_value=0)
    l.drop(columns = 'Unknown',inplace= True)
    l['Total'] = 0
    for i in l.columns[l.columns != 'Total']:
        l['Total'] += l[i]

    for i in l.columns[l.columns != 'Total']:
        l['%' + i] = 100*l[i]/l['Total']
    return l

In [432]:
type_by_year(50,1800)

Type,Boating,Invalid,Provoked,Sea Disaster,Unprovoked,Total,%Boating,%Invalid,%Provoked,%Sea Disaster,%Unprovoked
Year,,,,,,,,,,,
"(1800, 1850]",3,9,2,6,68,88,3.409091,10.227273,2.272727,6.818182,77.272727
"(1850, 1900]",18,47,25,36,275,401,4.488778,11.720698,6.234414,8.977556,68.578554
"(1900, 1950]",60,92,103,84,627,966,6.211180,9.523810,10.662526,8.695652,64.906832
"(1950, 2000]",129,193,256,78,1661,2317,5.567544,8.329737,11.048770,3.366422,71.687527
"(2000, 2050]",100,167,156,15,1535,1973,5.068424,8.464268,7.906741,0.760264,77.800304
